In [1]:
import torch
import numpy as np
import scipy
from scipy.stats import ortho_group
import scipy.linalg
from models import NeuralNet, CNN, CNN_FC, CNN_OneD
import torch
from torch import nn
from advertorch.attacks import LinfPGDAttack, L2PGDAttack, DDNL2Attack
import ipdb
import itertools
# from ipynb.fs.full.orthogonal_vectors import train, test_pgd


In [2]:
def train(model, data, labels, epochs = 1000):
    model.train()
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
    batch_size = data.shape[0]
    for epoch in range(epochs):
        if epoch==epochs//2:
            for param_group in optimizer.param_groups:
                param_group['lr'] /=10
        correct = 0
        total_loss = 0
#         ipdb.set_trace()
        for idx in range(data.shape[0]//batch_size):
            batch_data, batch_labels = data[idx*batch_size:(idx+1)*batch_size], labels[idx*batch_size:(idx+1)*batch_size]
            out = model(batch_data)
            loss = criterion(out, batch_labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
#             ipdb.set_trace()
            pred = out.argmax(1)
            correct += (pred == batch_labels).sum().item()
            total_loss += loss.item()*batch_size
        if epoch%(epochs/5)==0:
            print("Epoch: {}, Accuracy: {:.2f}, Loss: {:.5f}".format(epoch, 100*correct/data.shape[0], total_loss/data.shape[0]))
    return model

def test_pgd(model, data, labels, epsilon, res_row):
    model.eval()
    criterion = nn.CrossEntropyLoss(reduction='sum')
    adversary = L2PGDAttack(model, loss_fn=criterion, nb_iter=100, eps_iter=epsilon/50,
                                rand_init=True, eps=epsilon, clip_min=data.min().item(), clip_max=data.max().item(), targeted=False)
    # Set requires_grad attribute of tensor. Important for Attack
    data.requires_grad = True
#     ipdb.set_trace()
    perturbed_data = adversary.perturb(data, labels)
    new_out = model(perturbed_data)
#     ipdb.set_trace()
    pred = new_out.argmax(1)
    correct = (pred==labels).sum()
    res_row.append(100*float(correct)/labels.shape[0])
#     print("Epsilon: {}, Accuracy: {}".format(epsilon, 100*float(correct)/labels.shape[0]))
    
def test_ddn(model, data, labels, res_row):
    model.eval()
    criterion = nn.CrossEntropyLoss(reduction='sum')
    adversary = DDNL2Attack(model, nb_iter=1000, quantize = False, clip_min=data.min().item(), clip_max=data.max().item(),
                           loss_fn=criterion,)
#     adversary = L2PGDAttack(model, loss_fn=nn.MSELoss(reduction="sum"), nb_iter=40, eps_iter=epsilon/20,
#                                 rand_init=True, eps=epsilon, clip_min=data.min().item(), clip_max=data.max().item(), targeted=False)
    # Set requires_grad attribute of tensor. Important for Attack
    data.requires_grad = True
#     ipdb.set_trace()
    perturbed_data = adversary.perturb(data, labels)
    new_out = model(perturbed_data)
    pred = new_out.argmax(1)
    correct = (pred==labels).sum()
#     ipdb.set_trace()
    l2_distances = np.linalg.norm((data - perturbed_data).detach().cpu().numpy(),
                                  ord=2, axis=1)
    res_row.append(np.mean(l2_distances))
    res_row.append(np.median(l2_distances))
#     print("Samples: {}, Median L2: {},"
#           "Mean L2: {}, Accuracy: {}".format(l2_distances.shape[0], l2_distances.median(),
#                                              l2_distances.mean(), 100*float(correct)/labels.shape[0]))
  

In [3]:
# np.random.ran

In [4]:
def orthogonal_frequencies(N, d=1000):
    x = np.linspace(0, 1, d)
#     import ipdb
#     ipdb.set_trace()
    X = np.array([], dtype=np.float64).reshape((0, d))
    for k in range(1, N//2+1):
        vec_sin = np.sin(2*k*np.pi*x)
        vec_sin /= np.linalg.norm(vec_sin, ord=2)
        vec_cos = np.cos(2*k*np.pi*x)
        vec_cos /= np.linalg.norm(vec_cos, ord=2)
        X = np.vstack([vec_sin, vec_cos, X])
    for k in range(N//2+1, N+1):
        vec_sin = np.sin(2*k*np.pi*x)
        vec_sin /= np.linalg.norm(vec_sin, ord=2)
        vec_cos = np.cos(2*k*np.pi*x)
        vec_cos /= np.linalg.norm(vec_cos, ord=2)
        X = np.vstack([vec_sin, vec_cos, X])
    return X

def orthogonal_frequencies_odd_even(N, d=1000):
    x = np.linspace(0, 1, d)
#     import ipdb
#     ipdb.set_trace()
    X = np.array([], dtype=np.float64).reshape((0, d))
    for k in range(1, N, 2):
        vec_sin = np.sin(2*k*np.pi*x)
        vec_sin /= np.linalg.norm(vec_sin, ord=2)
        vec_cos = np.cos(2*k*np.pi*x)
        vec_cos /= np.linalg.norm(vec_cos, ord=2)
        X = np.vstack([vec_sin, vec_cos, X])
    for k in range(2, N+1, 2):
        vec_sin = np.sin(2*k*np.pi*x)
        vec_sin /= np.linalg.norm(vec_sin, ord=2)
        vec_cos = np.cos(2*k*np.pi*x)
        vec_cos /= np.linalg.norm(vec_cos, ord=2)
        X = np.vstack([vec_sin, vec_cos, X])
    return X

def orthogonal_frequencies_random(N, d=1000):
    x = np.linspace(0, 1, d)
#     import ipdb
#     ipdb.set_trace()
    X = np.array([], dtype=np.float64).reshape((0, d))
    random_permutation = np.random.permutation(np.arange(1, N+1))
    for k in random_permutation:
        vec_sin = np.sin(2*k*np.pi*x)
        vec_sin /= np.linalg.norm(vec_sin, ord=2)
        vec_cos = np.cos(2*k*np.pi*x)
        vec_cos /= np.linalg.norm(vec_cos, ord=2)
        X = np.vstack([vec_sin, vec_cos, X])

    return X

In [58]:
# samples = [50, 100, 150, 200, 250, 300, 400, 450]
# samples = [200, 300, 400, 450]
samples = [50, 100, 200]
# samples = [100]
input_shape = 4000
data_all = []
for k in samples:
    res_row = []
#     X = orthogonal_frequencies(k, input_shape)
    X = orthogonal_frequencies_odd_even(k, input_shape)
#     X = orthogonal_frequencies_random(k, input_shape)
    d1, d2 = X[:k], X[k:]
#     d1, d2 = x[:k],x[input_shape//2:input_shape//2+k]
#     d1, d2 = add_linear_seperator(d1, d2, p)
#         d1, d2 = x[:k], -x[:k]
    data = torch.tensor(np.concatenate((d1, d2))).float()
    labels = torch.tensor(np.concatenate(([0]*d1.shape[0], [1]*d2.shape[0])))
    idx_random = np.random.permutation(data.shape[0])
    data, labels = data[idx_random], labels[idx_random]
    print("{} Dimension, {} Samples per class".format(input_shape, k))
#     model = NeuralNet(input_shape, 1, 100, 2, no_final=True)
    model = CNN_OneD(hidden_channels=128, kernel_size=input_shape//2,
                         num_classes = 2, no_final=True)
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = model.to(device)
    data = data.to(device)
    labels = labels.to(device)
    model = train(model, data, labels, 2000)
#     print("-----------------------")
#     print("-----------------------")
#     for epsilon in [0.04, 0.06, 0.07, 0.08, 0.1, 0.15, 0.2]:
#         test_pgd(model, data, labels, epsilon, res_row)
#     print("-----------------------")
#     print("-----------------------")
    test_ddn(model, data, labels, res_row)
    data_all.append(res_row)
    print(','.join(['{:.5f}'.format(x) for x in res_row]))
    print("-----------------------")

4000 Dimension, 50 Samples per class
Epoch: 0, Accuracy: 50.00, Loss: 0.69335
Epoch: 400, Accuracy: 100.00, Loss: 0.00010
Epoch: 800, Accuracy: 100.00, Loss: 0.00003
Epoch: 1200, Accuracy: 100.00, Loss: 0.00002
Epoch: 1600, Accuracy: 100.00, Loss: 0.00002
0.67422,0.33807
-----------------------
4000 Dimension, 100 Samples per class
Epoch: 0, Accuracy: 50.00, Loss: 0.69359
Epoch: 400, Accuracy: 100.00, Loss: 0.00013
Epoch: 800, Accuracy: 100.00, Loss: 0.00004
Epoch: 1200, Accuracy: 100.00, Loss: 0.00003
Epoch: 1600, Accuracy: 100.00, Loss: 0.00002
0.76736,0.32183
-----------------------
4000 Dimension, 200 Samples per class
Epoch: 0, Accuracy: 50.00, Loss: 0.69343
Epoch: 400, Accuracy: 100.00, Loss: 0.00019
Epoch: 800, Accuracy: 100.00, Loss: 0.00006
Epoch: 1200, Accuracy: 100.00, Loss: 0.00004
Epoch: 1600, Accuracy: 100.00, Loss: 0.00003
0.83469,0.30428
-----------------------


FC
0.11425,0.09780
0.09180,0.08079
0.07655,0.07280

In [14]:
X

array([[ 0.00000000e+00,  1.86000927e-02,  3.00887079e-02, ...,
        -3.00887079e-02, -1.86000927e-02,  2.48539593e-16],
       [ 3.16148739e-02,  2.55711281e-02,  9.75062993e-03, ...,
         9.75062993e-03,  2.55711281e-02,  3.16148739e-02],
       [ 0.00000000e+00,  1.84388973e-02,  2.99636804e-02, ...,
        -2.99636804e-02, -1.84388973e-02,  1.61253138e-15],
       ...,
       [ 3.16148739e-02,  3.16134684e-02,  3.16092519e-02, ...,
         3.16092519e-02,  3.16134684e-02,  3.16148739e-02],
       [ 0.00000000e+00,  9.94202750e-05,  1.98839568e-04, ...,
        -1.98839568e-04, -9.94202750e-05, -7.74728349e-18],
       [ 3.16148739e-02,  3.16147177e-02,  3.16142492e-02, ...,
         3.16142492e-02,  3.16147177e-02,  3.16148739e-02]])

In [51]:
X = orthogonal_frequencies(50, 4000)

In [52]:
X.shape

(100, 4000)

In [53]:
max_dot = -1
i, j = -1, -1
for i in range(1,50):
    for j in range(1, 50):
        if i==j:
            continue
        curr_dot = np.dot(X[i], X[j])
        if curr_dot > max_dot:
            max_dot = curr_dot
            max_i, max_j = i, j

In [54]:
print(max_dot)

0.0004998750312485997


In [55]:
max_i

1

In [56]:
max_j

3